In [15]:
import earthaccess
import elevation
from pyhdf.SD import SD, SDC
import xarray as xr
import rasterio as rio
import glob
from rasterio.transform import from_origin
from rasterio.warp import reproject, Resampling
import geopandas as geo
import numpy as np
import os
from rasterio.mask import mask

In [5]:
# authenticate with Earthdata
earthaccess.login(strategy="netrc")

In [ ]:
# modis setup
hdf_file = r"C:\Users\bzwil\OneDrive\Desktop\KaliningradProject\kaliningrad-geoint-change-detection\files\data\modis_hdf\MOD13Q1.A2024081.h19v03.061.2024099105615.hdf"

# parsing through hdf file 
hdf = SD(hdf_file, SDC.READ)

print("datasets:")
for name, desc in hdf.datasets().items():
    print(name)
    
# extract NDVI dataset
ndvi_data = hdf.select('250m 16 days NDVI').get().astype('float32')
ndvi_data[ndvi_data == -3000] = np.nan # obviating unknown values
ndvi_data /= 10000.0
print("complete")

In [ ]:
# MOD13Q1 (vegetaion indices 16-day L3 global 250m)
granules = earthaccess.search_data(
    short_name = "MOD13Q1",
    temporal = ("2024-04-01", "2024-04-30"), 
    bounding_box = (19.8, 54.3, 21.5, 55.1), 
    cloud_hosted = True
)

download_files = earthaccess.download(granules[:1])
print(download_files)

# situate the ndvi file properly
ndvi_path = os.path.join(hdf_file, "..", "..", "ndvi", "ndvi_output.tif")

with rio.open(ndvi_path) as ndvi:
    ndvi_data = ndvi.read(1).astype('float32')
    ndvi_data[ndvi_data == -3000] = float('nan')
    ndvi_data = ndvi_data / 10000.0 
    
    print("NDVI shape:", ndvi_data.shape)

In [ ]:
# VIIRS fire data
path = r"C:\Users\bzwil\OneDrive\Desktop\KaliningradProject\kaliningrad-geoint-change-detection\files\data\viirs_nc"

nc_files = glob.glob(os.path.join(path, "*.nc"))
target_variable = 'fire mask' # paramount data

# processing each .nc file & matching file shape
for nc_file in nc_files:
    try:
        ds = xr.open_dataset(nc_file)
        
        fire_mask = ds[target_variable].values  
        lat = ds['FP_latitude'].values         
        lon = ds['FP_longitude'].values       
        print("Variables extracted: fire mask, latitude, longitude", flush=True)

        if fire_mask.shape != lat.shape or fire_mask.shape != lon.shape:
            rows, cols = fire_mask.shape
        else:
            rows, cols = fire_mask.shape
        
        nodata_value = 255 if fire_mask.dtype == 'uint8' else -9999     
               
        lon_min, lon_max = lon.min(), lon.max()
        lat_min, lat_max = lat.min(), lat.max()
        rows, cols = fire_mask.shape
        x_res = (lon_max - lon_min) / cols
        y_res = (lat_max - lat_min) / rows

        transform = from_origin(lon_min, lat_max, x_res, y_res)

        metadata = {
            'driver': 'GTiff',
            'height': rows,
            'width': cols,
            'count': 1,  
            'dtype': fire_mask.dtype,
            'crs': 'EPSG:4326',  
            'transform': transform,
            'nodata': nodata_value  
        }

        # creating output filename
        output_tif = os.path.join(path, os.path.splitext(os.path.basename(nc_file))[0] + f"_{target_variable.replace(' ', '_')}.tif")
        print(f"Output file will be: {output_tif}", flush=True)

        # writing geotiff
        with rasterio.open(output_tif, 'w', **metadata) as dst:
            dst.write(fire_mask, 1)  # Write as band 1
            print(f"Saved GeoTIFF: {output_tif}", flush=True)

        ds.close()
    except Exception as e:
        print(f"Error processing {nc_file}: {e}", flush=True)
        continue

In [ ]:
# NASA-SRTM DEM
# run via Ubuntu

bounds = (19.5, 54.3, 22.9, 55.5)
dem = "/mnt/c/Users/bzwil/OneDrive/Desktop/KaliningradProject/kaliningrad-geoint-change-detection/files/data/dem/kaliningrad_dem.tif"
elevation.clip(bounds=bounds, output=dem)

# ensuring all data is scaled properly
with rio.open(dem) as src:
    dem_raw = src.read(1).astype("float32")

    if src.nodata is not None:
        dem_raw[dem_raw == src.nodata] = np.nan
        
dem_raw[(dem_raw < -5000) | (dem_raw > 5000)] = np.nan    

In [ ]:
# ensuring alignment with aoi
general_path = r"C:\Users\bzwil\OneDrive\Desktop\KaliningradProject\kaliningrad-geoint-change-detection\files\data"

aoi_path = os.path.join(general_path, r"kaliningrad-oblast.geojson")
ndvi_path = os.path.join(general_path, r"ndvi\ndvi_output.tif")
fire_mask_path = os.path.join(general_path, r"viirs_nc\VNP14IMG.A2024092.0036.002.2024093145722_fire_mask.tif")
dem_path = os.path.join(general_path, r"dem\kaliningrad_srtm.tif")

output_dir = r"C:\Users\bzwil\OneDrive\Desktop\KaliningradProject\kaliningrad-geoint-change-detection\files\data\prepared_data"
os.makedirs(output_dir, exist_ok=True)

aoi = geo.read_file(aoi_path)

# function to clip aoi
def clip_to_aoi (raster_path, aoi, out_name):
    with rio.open(raster_path) as src:
        if aoi.crs != src.crs:
            aoi = aoi.to_crs(src.crs)
        geoms = [feature["geometry"] for feature in aoi.__geo_interface__["features"]]
        
        out_image, out_transform = mask(src, geoms, crop=True)
        out_meta = src.meta.copy()
        
        out_meta.update({
            "driver": "GTiff", 
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
        
        out_path = os.path.join(output_dir, out_name)
        with rio.open(out_path, "w", **out_meta) as dest:
            dest.write(out_image)
            
        return out_image[0], out_meta

# function to ensure matrix alignment
def reproject_match(src_array, src_meta, target_meta):
    dst_array = np.empty((1, target_meta["height"], target_meta["width"]), dtype=src_meta["dtype"])
    
    reproject(
        source=src_array,
        destination=dst_array,
        src_transform=src_meta["transform"],
        src_crs=src_meta["crs"],
        dst_transform=target_meta["transform"],
        dst_crs=target_meta["crs"],
        resampling=Resampling.nearest
    )

    return dst_array[0]    

ndvi_data, ndvi_meta = clip_to_aoi(ndvi_path, aoi, "ndvi_clipped.tif")
fire_data, fire_meta = clip_to_aoi(fire_mask_path, aoi, "fire_mask_clipped.tif")
dem_data, dem_meta = clip_to_aoi(dem_path, aoi, "dem_clipped.tif")

fire_data_aligned = reproject_match(fire_data[np.newaxis, ...], fire_meta, ndvi_meta)
dem_data_aligned = reproject_match(dem_data[np.newaxis, ...], dem_meta, ndvi_meta)


 All rasters are now aligned to NDVI grid.
